In [23]:
import numpy as np 
import pandas as pd 
import cv2
import os 
import SimpleITK as sitk
from tfmed.utils.med_ops import *

In [24]:
qiguan = 'red'
pre_qiguan = 'p_red'
data_number = '7'
data_size = 128

In [25]:
cls_out = 'val_label_info_new2.csv'
# cls_out = 'test_info_add_cls_update_all.csv'

data_test = pd.read_csv(cls_out)
data_test.head()

,filename,maskname,red,pink,yellow,blue,p_red,p_pink,p_yellow,p_blue,folder,y_s,y_e,x_s,x_e
0,/public/lixin/SegTHOR/input/train_fla/validate...,/public/lixin/SegTHOR/input/train_fla/validate...,1,1,0,1,1.0,1.0,0.0,1.0,3,1,508,144,505
1,/public/lixin/SegTHOR/input/train_fla/validate...,/public/lixin/SegTHOR/input/train_fla/validate...,1,1,0,1,1.0,1.0,0.0,1.0,3,1,508,144,505
2,/public/lixin/SegTHOR/input/train_fla/validate...,/public/lixin/SegTHOR/input/train_fla/validate...,1,0,1,1,1.0,0.0,1.0,1.0,3,1,508,144,505
3,/public/lixin/SegTHOR/input/train_fla/validate...,/public/lixin/SegTHOR/input/train_fla/validate...,1,1,0,1,1.0,1.0,0.0,1.0,3,1,508,144,505
4,/public/lixin/SegTHOR/input/train_fla/validate...,/public/lixin/SegTHOR/input/train_fla/validate...,1,1,0,1,1.0,1.0,0.0,1.0,3,1,508,144,505


In [27]:
red_model = 

In [29]:
from segmentation_models import Unet
BACKBONE = 'densenet121'
model = Unet(BACKBONE, encoder_weights=None)
from keras.utils import multi_gpu_model ##

model = multi_gpu_model(model,gpus=2) ##
model.load_weights(red_model)

In [30]:
f = 34
df = data_test.loc[data_test['folder'] == f] 
path = df['filename'].values[0]
folder = '/'.join(path.split('/')[:-1])
print(folder)
length = len(df)

mk = []
for i in range(length):
    image = os.path.join(folder, 'Patient_' + str(f).zfill(2) +'_' + str(i) + '.png')

    ys = data_test.loc[data_test['filename'] == image]['y_s'].values[0]
    ye = data_test.loc[data_test['filename'] == image]['y_e'].values[0]
    xs = data_test.loc[data_test['filename'] == image]['x_s'].values[0]
    xe = data_test.loc[data_test['filename'] == image]['x_e'].values[0]
    
    image7 = os.path.join('/public/lixin/SegTHOR/input/train_fla' + str(data_number) + '/validates', str(f).zfill(2), 'images', image.split('/')[-1])
    
    print(image7)
    c_img = crop_pic(cv2.imread(image7),ys = ys,ye = ye,xs = xs,xe = xe)
    first_img = np.expand_dims(c_img, 0)/255.0

    
    
    red_cls = data_test.loc[data_test['filename'] == image][qiguan].values[0]

    if red_cls == 1.0:
        out_red = model.predict(first_img)[0,:,:,0] #np.round()
        pred = np.zeros((512,512))
        pred[int(np.ceil((xe+xs-70)/2))- data_size: int(np.ceil((xe+xs-70)/2))+data_size, int(np.ceil((ye+ys)/2))-data_size: int(np.ceil((ye+ys)/2))+data_size] = out_red
    else:
        pred = np.zeros((512, 512))
    mk.append(pred.tolist())
    
    
mk = np.stack(mk, 0)
mk = np.array(mk,dtype=np.float32)

ori_path = os.path.join('/public/lixin/SegTHOR/input/train/','Patient_' + str(f).zfill(2),'Patient_' + str(f).zfill(2) + '.nii.gz') #Patient_ str(f).zfill(2)
print(ori_path)

o_folder = os.path.join('/public/lixin/SegTHOR/segthor/2d_try/sample/',qiguan)

out_path = os.path.join(o_folder, 'Patient_'+str(f).zfill(2) +'.nii.gz')

WriteNiiFromArray(mk, out_path, **GetSettings(ori_path))


/public/lixin/SegTHOR/input/train_fla/validates/34/images
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_0.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_1.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_2.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_3.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_4.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_5.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_6.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_7.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_8.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_9.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_10.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_11.png
/public/lixin/SegTHOR/input/

/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_107.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_108.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_109.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_110.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_111.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_112.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_113.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_114.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_115.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_116.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_117.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patient_34_118.png
/public/lixin/SegTHOR/input/train_fla7/validates/34/images/Patie